In [1]:
import os
os.chdir('..')

In [2]:
os.environ['UCX_TLS'] = 'tcp,cuda_copy'

In [3]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client


cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0,1",
    protocol="ucx",
    enable_tcp_over_ucx=True,
    enable_infiniband=True,
    rmm_managed_memory=True,
    rmm_pool_size='24GB'
)
client = Client(cluster)
client

/usr/local/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33953 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:33953/status,
Dashboard: http://127.0.0.1:33953/status,Workers: 2
Total threads: 2,Total memory: 502.58 GiB
Status: running,Using processes: True
Comm: ucx://127.0.0.1:57449,Workers: 2
Dashboard: http://127.0.0.1:33953/status,Total threads: 2
Started: Just now,Total memory: 502.58 GiB
Comm: ucx://127.0.0.1:35465,Total threads: 1
Dashboard: http://127.0.0.1:42961/status,Memory: 251.29 GiB
Nanny: ucx://127.0.0.1:37925,


In [4]:
# import cudf as pd
import dask
import dask_cudf
import dask.dataframe as dd
import cudf, cupy

dask.config.set({"dataframe.backend": "cudf"})

In [5]:
meta_tower = dd.read_csv('data/processed/meta_tower.csv')

meta_tower

,cell_id,lon,lat,deyang_center_flag,tower_area_code
npartitions=1,,,,,
,object,float64,float64,int64,int64
,...,...,...,...,...


In [6]:
meta_tower.size.compute()

202845

In [7]:
import hvplot.cudf

In [9]:
"""
tiles:
'CartoDark', 'CartoEco', 'CartoLight', 'CartoMidnight', 'EsriAntarcticImagery', 'EsriArcticImagery', 'EsriArcticOceanBase', 
'EsriArcticOceanReference', 'EsriDelormeWorldBaseMap', 'EsriImagery', 'EsriNatGeo', 'EsriOceanBase', 'EsriOceanReference', 
'EsriReference', 'EsriTerrain', 'EsriUSATopo', 'EsriWorldBoundariesAndPlaces', 'EsriWorldBoundariesAndPlacesAlternate', '
EsriWorldDarkGrayBase', 'EsriWorldDarkGrayReference', 'EsriWorldHillshade', 'EsriWorldHillshadeDark', 'EsriWorldLightGrayBase', 
'EsriWorldLightGrayReference', 'EsriWorldNavigationCharts', 'EsriWorldPhysical', 'EsriWorldShadedRelief', 'EsriWorldStreetMap', 
'EsriWorldTopo', 'EsriWorldTransportation', 'OSM', 'OpenTopoMap'
"""

(
    meta_tower
    .compute()
    .hvplot.points('lon', 'lat', geo=True, color='red', alpha=0.05, tiles='EsriWorldStreetMap')
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]

In [12]:
(
    meta_tower[(meta_tower['tower_area_code'] == 838)]
    .compute()
    .hvplot
    .points('lon', 'lat', geo=True, color='deyang_center_flag', alpha=0.2, tiles='EsriWorldStreetMap')
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]   (deyang_center_flag)

# Popularity

In [20]:
from dask_cudf import read_parquet
df = read_parquet('data/processed/201308/userinfo_cdr')


df.head()

,serv_id,client_nbr,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id,lon,lat,deyang_center_flag,tower_area_code
0,92vb7ake,e4dc8jvr,8,5,e4dc8jvr,838,117,834,efpu3bzv,34B8,104.404,31.1291,1,838
1,9mjki07k,ec348kkr,8,5,ec348kkr,838,55,838,8qvk4x65,3459,104.374,31.1269,1,838
2,apcvf87m,fu3k8k9o,8,5,fu3k8k9o,838,23,838,6694ksgr,36A6,104.412,31.1209,1,838
3,9xtc9fyi,ez8g8kcf,8,5,4s6rgij3,835,23,838,ez8g8kcf,3C6A,104.381,31.0947,1,838
4,9fd9n5pl,m1v48ki3,8,5,msts3mez,838,26,838,m1v48ki3,36A2,104.384,31.1522,1,838


In [21]:
df.shape[0].compute()

51843831

## tower popularity rank in all cdr

In [22]:
rank = cudf.from_pandas(
     df
    .groupby(['cell_id'])
    .size()
    .compute()
    .to_pandas()
    .rank(pct=True)
    .to_frame('popularity_rank')
 )

popularity_rank_plot = (
    cudf
    .merge(rank, meta_tower.compute(), on='cell_id')
    .hvplot
    .points('lon', 'lat', geo=True, color='popularity_rank', alpha=0.2, tiles='EsriWorldStreetMap')
)

popularity_rank_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]   (popularity_rank)

## tower popularity rank within 838(deyang)

In [23]:
rank = cudf.from_pandas(
     df[df['tower_area_code'] == 838]
    .groupby(['cell_id'])
    .size()
    .compute()
    .to_pandas()
    .rank(pct=True)
    .to_frame('popularity_rank')
 )

popularity_rank_plot = (
    cudf
    .merge(rank, meta_tower.compute(), on='cell_id')
    .hvplot
    .points('lon', 'lat', geo=True, color='popularity_rank', alpha=0.2, tiles='EsriWorldStreetMap')
)

popularity_rank_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]   (popularity_rank)

## output

In [12]:
import geckodriver_autoinstaller

geckodriver_autoinstaller.install(cwd=True)

'/home/r12323011/telecom/v0.33.0/geckodriver'

In [15]:
import holoviews as hv
from bokeh.io import export_svgs

plot = hv.render(popularity_rank_plot)
plot.output_backend = "svg"
export_svgs(plot, filename="report/plot/tower/popularity_rank_201308.svg")

['report/plot/tower/popularity_rank_201308.svg']

## cdr not in deyang is calling or called?

In [24]:
target = df[
    (df['tower_area_code'] != 838) &
    (
        (df['called_area_code'] == df['tower_area_code']) |
        (df['calling_area_code'] == df['tower_area_code'])
    )
]
target.shape[0].compute()

1607128

In [25]:
target[target['calling_area_code'] == 838].shape[0].compute()

892070

In [26]:
target[target['called_area_code'] == 838].shape[0].compute()

715058

## cdr not in deyang but in chengdu is calling or called?

In [27]:
target = df[
    (df['tower_area_code'] == 28) &
    (
        (df['called_area_code'] == df['tower_area_code']) |
        (df['calling_area_code'] == df['tower_area_code'])
    )
]
target.shape[0].compute()

1170607

In [28]:
target[target['calling_area_code'] == 28].shape[0].compute()

516828

In [29]:
target[target['called_area_code'] == 28].shape[0].compute()

653779